## 图像识别-生活垃圾分类
### 迁移学习 Transfer Learning

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import keras
from keras.layers import Dense,Flatten, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, BatchNormalization, Input
from keras.applications.densenet import DenseNet201, preprocess_input
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import np_utils, to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras import regularizers

plt.style.use('ggplot')
plt.switch_backend('agg')

我们需要在网络的最后一层有尽可能多的神经元，一遍我们得到和我们想要识别的图片类别的数量一样。 因此，我们删掉这个了1000个神经元的网络层，并在网络最后添加了我们自己的定义的全连接层。

这过程可以通过导入模型时设置（IncludeTop=False)来实现。

我们最后一层只需要16个神经元(16大类生活垃圾)。使用以下代码完成。


### 导入和构建所需模型

现在我们有了模型，我们将使用预先训练的权重，我们的模型已经在(Imagenet数据集)上训练过了，我们必须将所有的权重设置为不可训练。我们将只训练我们之前制作的最后一层致密层。

DenseNet-121：其中，k=32，k=48中的k是growth rate，表示输出feature map的个数。DenseNet-121中121是层的个数（卷积层+全连接层），在这里就是(6+12+24+16)*2+1(7*7conv)+3(Translation layer)+1(fc)=121.

In [3]:
# 分类个数
NUM_CLASSES = 16

# 进行训练和测试的图片大小
IMAGE_SIZE = 192

# create the base pre-trained model
Inp = Input((IMAGE_SIZE, IMAGE_SIZE, 3))

base_model = DenseNet201(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, weights='imagenet')
#model = load_model('./Amodels/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False) # 加载本地模型

# 自定义FC层以基本模型的输入为卷积层的最后一层
x = base_model(Inp)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(2048, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
x = BatchNormalization()(x)
x = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
x = BatchNormalization()(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=Inp, outputs=predictions)

# 可观察模型结构
model.summary()

# 获取模型的层数
print('layer nums:', len(model.layers))

W0301 19:05:21.817187  5512 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0301 19:05:21.849189  5512 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0301 19:05:21.858189  5512 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0301 19:05:21.883191  5512 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0301 19:05:21.886193  5512 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\keras\backend\ten

74842112/74836368 [==============================] - 377s 5us/step


W0301 19:12:31.002200  5512 deprecation.py:506] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 192, 192, 3)       0         
_________________________________________________________________
densenet201 (Model)          (None, 6, 6, 1920)        18321984  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1920)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1920)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              3934208   
_________________________________________________________________
batch_normalization_1 (Batch (None, 2048)              8192      
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              2098176   
__________

In [4]:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

0 input_2
1 zero_padding2d_1
2 conv1/conv
3 conv1/bn
4 conv1/relu
5 zero_padding2d_2
6 pool1
7 conv2_block1_0_bn
8 conv2_block1_0_relu
9 conv2_block1_1_conv
10 conv2_block1_1_bn
11 conv2_block1_1_relu
12 conv2_block1_2_conv
13 conv2_block1_concat
14 conv2_block2_0_bn
15 conv2_block2_0_relu
16 conv2_block2_1_conv
17 conv2_block2_1_bn
18 conv2_block2_1_relu
19 conv2_block2_2_conv
20 conv2_block2_concat
21 conv2_block3_0_bn
22 conv2_block3_0_relu
23 conv2_block3_1_conv
24 conv2_block3_1_bn
25 conv2_block3_1_relu
26 conv2_block3_2_conv
27 conv2_block3_concat
28 conv2_block4_0_bn
29 conv2_block4_0_relu
30 conv2_block4_1_conv
31 conv2_block4_1_bn
32 conv2_block4_1_relu
33 conv2_block4_2_conv
34 conv2_block4_concat
35 conv2_block5_0_bn
36 conv2_block5_0_relu
37 conv2_block5_1_conv
38 conv2_block5_1_bn
39 conv2_block5_1_relu
40 conv2_block5_2_conv
41 conv2_block5_concat
42 conv2_block6_0_bn
43 conv2_block6_0_relu
44 conv2_block6_1_conv
45 conv2_block6_1_bn
46 conv2_block6_1_relu
47 conv2_block

#### 冻结和微调

In [5]:
# # 全部冻结
# for layer in model.layers:
#     layer.trainable=False


# 冻结训练的层数，根据模型的不同，层数也不一样，根据调试的结果，最后一个卷积块参与训练
# 除了FC层，靠近FC层的一部分卷积层可参与参数训练，一般来说，模型结构已经标明一个卷积块包含的层数 (最后一个卷积块和FC层要参与参数训练)
FREEZE_LAYERS =  481 
for layer in base_model.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in base_model.layers[FREEZE_LAYERS:]:
    layer.trainable = True
for layer in base_model.layers:
    print(layer.name, layer.trainable)

input_2 False
zero_padding2d_1 False
conv1/conv False
conv1/bn False
conv1/relu False
zero_padding2d_2 False
pool1 False
conv2_block1_0_bn False
conv2_block1_0_relu False
conv2_block1_1_conv False
conv2_block1_1_bn False
conv2_block1_1_relu False
conv2_block1_2_conv False
conv2_block1_concat False
conv2_block2_0_bn False
conv2_block2_0_relu False
conv2_block2_1_conv False
conv2_block2_1_bn False
conv2_block2_1_relu False
conv2_block2_2_conv False
conv2_block2_concat False
conv2_block3_0_bn False
conv2_block3_0_relu False
conv2_block3_1_conv False
conv2_block3_1_bn False
conv2_block3_1_relu False
conv2_block3_2_conv False
conv2_block3_concat False
conv2_block4_0_bn False
conv2_block4_0_relu False
conv2_block4_1_conv False
conv2_block4_1_bn False
conv2_block4_1_relu False
conv2_block4_2_conv False
conv2_block4_concat False
conv2_block5_0_bn False
conv2_block5_0_relu False
conv2_block5_1_conv False
conv2_block5_1_bn False
conv2_block5_1_relu False
conv2_block5_2_conv False
conv2_block5_co

### 编译模型

In [6]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

W0301 19:15:16.344417  5512 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



### 加载数据集

In [7]:
#加载处理好的数据与标签
Xtrain = np.load('encoded/X_train_C_192.npy')
Xvalid = np.load('encoded/X_valid_C_192.npy')
Ytrain = np.load('encoded/Y_train_C_192.npy')
Yvalid = np.load('encoded/Y_valid_C_192.npy')
print(Xtrain.shape)
print(Xvalid.shape)

(22002, 192, 192, 3)
(9430, 192, 192, 3)


### 训练模型

In [9]:
# 训练模型
epochs = 20
batch_size = 64

history = model.fit(Xtrain, Ytrain, 
                          epochs =epochs, 
                          shuffle = True, 
                          batch_size = batch_size,
                          validation_data=(Xvalid, Yvalid))  

Train on 22002 samples, validate on 9430 samples
Epoch 1/20
22002/22002 [==============================] - 13065s 594ms/step - loss: 1.3489 - acc: 0.6983 - val_loss: 0.8971 - val_acc: 0.7992
Epoch 2/20
22002/22002 [==============================] - 12619s 574ms/step - loss: 0.8207 - acc: 0.8177 - val_loss: 0.7139 - val_acc: 0.8508
Epoch 3/20
22002/22002 [==============================] - 12606s 573ms/step - loss: 0.6739 - acc: 0.8576 - val_loss: 0.8055 - val_acc: 0.8349
Epoch 4/20
22002/22002 [==============================] - 12594s 572ms/step - loss: 0.5829 - acc: 0.8854 - val_loss: 0.6818 - val_acc: 0.8615
Epoch 5/20
22002/22002 [==============================] - 12623s 574ms/step - loss: 0.4933 - acc: 0.9051 - val_loss: 0.7100 - val_acc: 0.8549
Epoch 6/20
22002/22002 [==============================] - 12615s 573ms/step - loss: 0.4266 - acc: 0.9190 - val_loss: 0.6625 - val_acc: 0.8638
Epoch 7/20
22002/22002 [==============================] - 12628s 574ms/step - loss: 0.3630 - acc: 0

### 训练过程可视化

In [10]:
print(history.history)
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs_range = range(epochs)
 
plt.figure(figsize=(8, 8))
 
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
 
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

{'val_loss': [0.8971064966278642, 0.7139283305514024, 0.8055228358235365, 0.681763061827339, 0.7099545467822701, 0.6624859670566223, 0.6680260076629015, 0.6973988079696041, 0.6076598361213539, 0.6112645003474529, 0.5403372174482457, 0.6437514045837701, 0.7405312270968994, 0.7109975717100586, 0.6323875104635164, 0.6271678137627538, 0.6599272530253341, 0.6182305976669458, 0.7815456717795557, 0.6572487412429437], 'val_acc': [0.799151643639784, 0.8507953341035044, 0.8348886532596415, 0.8615058324369873, 0.8549310710751239, 0.8638388123264494, 0.8621420996313004, 0.8528101802251499, 0.8746553552365631, 0.8762460233550815, 0.8858960763773509, 0.8745493106978569, 0.8497348887164419, 0.8611876988208685, 0.8797454931703125, 0.8767762459727638, 0.8740190880043256, 0.8831389183077807, 0.8550371155506227, 0.8801696712492885], 'loss': [1.3488719059482357, 0.820711358158537, 0.6738734446528954, 0.5829172464687492, 0.49328890172755346, 0.4265971522842274, 0.36299675371189377, 0.3414326486002585, 0.28

KeyError: 'accuracy'

In [11]:
#6.7为可视化
def overfitting_plot(history, name):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs = range(1, len(acc) + 1)
    
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend(loc='best')
    plt.savefig('acc_'+name+'.png')
    
    plt.figure()
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend(loc='best')
    plt.savefig('loss_'+name+'.png')

In [12]:
overfitting_plot(history, '')

### 保存模型

In [14]:
# 保存模型
model.save('modelC_TR_DenseNet201_192.h5')

### 模型评估

In [13]:
num_classes = 16
y_pred = model.predict(Xvalid)
# Convert one-hot to index
y_pred = np.argmax(y_pred, axis=1)
y_pred = to_categorical(y_pred, num_classes)
from sklearn.metrics import classification_report
print(classification_report(Yvalid, y_pred))
#f1值指标

              precision    recall  f1-score   support

           0       0.80      0.90      0.84       541
           1       0.94      0.94      0.94       664
           2       0.91      0.91      0.91       477
           3       0.81      0.83      0.82       645
           4       0.89      0.92      0.91       531
           5       0.79      0.91      0.84       703
           6       0.89      0.82      0.85       621
           7       0.86      0.91      0.89       564
           8       0.91      0.97      0.94       676
           9       0.93      0.84      0.88       567
          10       0.93      0.89      0.91       656
          11       0.90      0.93      0.92       610
          12       0.82      0.97      0.89       473
          13       0.91      0.86      0.88       599
          14       0.95      0.69      0.80       612
          15       0.93      0.80      0.86       491

   micro avg       0.88      0.88      0.88      9430
   macro avg       0.88   